In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end
 
Pkg.build("PyCall")
using JuliaProbo
using Seaborn: heatmap, savefig
gr()

In [ ]:
function init_reward(pe::PolicyEvaluator, world::PuddleWorld, sampling_num::Int64)
    reso = pe.reso
    dxs = collect(range(0, reso[1], length=sampling_num))
    dys = collect(range(0, reso[2], length=sampling_num))
    index_nums = pe.index_nums
    pose_min = pe.pose_min
    puddles = world.puddles_
    reward = zeros(Float64, index_nums[1], index_nums[2])
    
    for x in 0:index_nums[1]-1
        for y in 0:index_nums[2]-1
            for dx in dxs
                for  dy in dys
                    pose = pose_min .+ (reso .* [x, y, 0]) .+ [dx, dy, 0]
                    for puddle in puddles
                        reward[x+1, y+1] += ( puddle.depth * ( convert(Float64, inside(puddle, pose)) ) )
                    end
                end
            end
            reward[x+1, y+1] /= (sampling_num)^2
        end
    end
    return reward
end

In [ ]:
xlim = [-5.0, 5.0]
ylim = [-5.0, 5.0]
world = PuddleWorld(xlim, ylim)
push!(world, Puddle([-2.0, 0.0], [0.0, 2.0], 0.1))
push!(world, Puddle([-0.5, -2.0], [2.5, 1.0], 0.1))

In [ ]:
sampling_num = 10
pe = PolicyEvaluator([0.1, 0.1, pi/20], Goal(-3.0, -3.0));
init_value(pe)
init_policy(pe)
init_state_transition_probs(pe, 0.1, sampling_num)
heatmap(rotl90(init_reward(pe, world, sampling_num)))
# savefig("ch10_policy_evaluation5.png")

<img src="ch10_policy_evaluation5.png">